In [4]:
import torch
from torch import nn

In [8]:
input = torch.randn(8, 3, 50, 100)
print(input.requires_grad)


net = nn.Sequential(nn.Conv2d(3, 16, 3, 1),
                    nn.Conv2d(16, 32, 3, 1))
for param in net.named_parameters():
    print(param[0], param[1].requires_grad)


output = net(input)
# 在写代码的过程中，不要把网络的输入和 Ground Truth 的 requires_grad 设置为 True。
# 虽然这样设置不会影响反向传播，但是需要额外计算网络的输入和 Ground Truth 的导数，增大了计算量和内存占用不说，
# 这些计算出来的导数结果也没啥用。因为我们只需要神经网络中的参数的导数，用来更新网络，其余的导数都不需要。

print(output.requires_grad)

False
0.weight True
0.bias True
1.weight True
1.bias True
True


In [11]:
input = torch.randn(8, 3, 50, 100)
print(input.requires_grad)

net = nn.Sequential(nn.Conv2d(3, 16, 3, 1),
                    nn.Conv2d(16, 32, 3, 1))
for param in net.named_parameters():
    param[1].requires_grad = False # 将梯度更新变成False，我们可以通过这种方法，在训练的过程中冻结部分网络，让这些层的参数不再更新，这在迁移学习中很有用处。
    print(param[0], param[1].requires_grad)

output = net(input)
print(output.requires_grad)


False
0.weight False
0.bias False
1.weight False
1.bias False
False


model = torchvision.models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

# 用一个新的 fc 层来取代之前的全连接层
# 因为新构建的 fc 层的参数默认 requires_grad=True
model.fc = nn.Linear(512, 100)

# 只更新 fc 层的参数
optimizer = optim.SGD(model.fc.parameters(), lr=1e-2, momentum=0.9)

# 通过这样，我们就冻结了 resnet 前边的所有层，
# 在训练过程中只更新最后的 fc 层中的参数。

In [12]:
# 当我们在做 evaluating 的时候（不需要计算导数），我们可以将推断（inference）的代码包裹在 with torch.no_grad(): 之中，
# 以达到 暂时 不追踪网络参数中的导数的目的，总之是为了减少可能存在的计算和内存消耗。
x = torch.randn(3, requires_grad = True)
print(x.requires_grad)
# True
print((x ** 2).requires_grad)
# True

with torch.no_grad():
    print((x ** 2).requires_grad)
    # False

print((x ** 2).requires_grad)
# True

True
True
False
True


In [24]:

# 在反向传播过程中，只有 is_leaf=True 的时候，需要求导的张量的导数结果才会被最后保留下来。对于 requires_grad=False 的 tensor 来说，我们约定俗成地把它们归为叶子张量。
# 但其实无论如何划分都没有影响，因为张量的 is_leaf 属性只有在需要求导的时候才有意义。



# 我们真正需要注意的是当 requires_grad=True 的时候，如何判断是否是叶子张量：

a = torch.ones([2, 2], requires_grad=True)
print(a.is_leaf,a.grad_fn)
# 当这个 tensor 是用户创建的时候，它是一个叶子节点,对于叶子节点来说，它们的 grad_fn 属性都为空；

b = a + 2
print(b.is_leaf,b.grad_fn)

# 当这个 tensor 是由其他运算操作产生的时候，它就不是一个叶子节点,因为 b 不是用户创建的，是通过计算生成的
#  而对于非叶子结点来说，因为它们是通过一些操作生成的，所以它们的 grad_fn 不为空。

True None
False <AddBackward0 object at 0x1277489a0>


In [27]:
'''inplace操作'''
# 情景 1
a = torch.tensor([3.0, 1.0])
print(id(a))
a = a.exp()
print(id(a))
# 在这个过程中 a.exp() 生成了一个新的对象，然后再让 a
# 指向它的地址，所以这不是个 inplace 操作

# 情景 2
a = torch.tensor([3.0, 1.0])
print(id(a)) 
a[0] = 10
print(id(a), a) # 2112716403840 tensor([10.,  1.])
# inplace 操作，内存地址没变

# 所以类似 i=i+1 不是inplace操作，而i+=1是inplace操作


5121600144
5121600864
5100578112
5100578112 tensor([10.,  1.])


In [ ]:
# pytorch通过._version来判断是否是inplace操作
# 每次 tensor 在进行 inplace 操作时，变量 _version 就会加1，其初始值为0。
# 在正向传播过程中，求导系统记录的 b 的 version 是0，但是在进行反向传播的过程中，求导系统发现 b 的 version 变成1了，所以就会报错了。
# 但是还有一种特殊情况不会报错，就是反向传播求导的时候如果没用到 b 的值（比如 y=x+1， y 关于 x 的导数是1，和 x 无关），自然就不会去对比 b 前后的 version 了，所以不会报错。


a = torch.tensor([1.0, 3.0], requires_grad=True)
b = a + 2
print(b._version) # 0

loss = (b * b).mean()
b[0] = 1000.0
print(b._version) # 1

loss.backward()

In [28]:
# 如果是grad_required = True 的叶子节点，inplace操作则会报错
# 本来是该有导数值保留的变量，现在成了导数会被自动释放的中间变量了，所以 PyTorch 就给你报错了。

a = torch.tensor([10., 5., 2., 3.], requires_grad=True)
print(a, a.is_leaf)
# tensor([10.,  5.,  2.,  3.], requires_grad=True) True

a[:] = 0
print(a, a.is_leaf)
# tensor([0., 0., 0., 0.], grad_fn=<CopySlices>) False

loss = (a*a).mean()
loss.backward()

tensor([10.,  5.,  2.,  3.], requires_grad=True) True


RuntimeError: a view of a leaf Variable that requires grad is being used in an in-place operation.

In [29]:
# 同样grad_required = True 的叶子节点，只要你对需要求导的叶子张量使用了这些操作，不需等到反向传播，马上就会报错。
a = torch.tensor([10., 5., 2., 3.], requires_grad=True)
a.add_(10.) # 或者 a += 10.

RuntimeError: a leaf Variable that requires grad is being used in an in-place operation.

In [ ]:
'''给叶子节点赋值的正确方法'''
#PyTorch 不推荐使用 inplace 操作，当求导过程中发现有 inplace 操作影响求导正确性的时候，会采用报错的方式提醒。
# 但这句话反过来说就是，因为只要有 inplace 操作不当就会报错，所以如果我们在程序中使用了 inplace 操作却没报错，那么说明我们最后求导的结果是正确的，没问题的。这就是我们常听见的没报错就没有问题。



# 方法一
a = torch.tensor([10., 5., 2., 3.], requires_grad=True)
print(a, a.is_leaf, id(a))
# tensor([10.,  5.,  2.,  3.], requires_grad=True) True 2501274822696

a.data.fill_(10.)
# 或者 a.detach().fill_(10.)
print(a, a.is_leaf, id(a))
# tensor([10., 10., 10., 10.], requires_grad=True) True 2501274822696

loss = (a*a).mean()
loss.backward()
print(a.grad)
# tensor([5., 5., 5., 5.])



# 方法二
a = torch.tensor([10., 5., 2., 3.], requires_grad=True)
print(a, a.is_leaf)
# tensor([10.,  5.,  2.,  3.], requires_grad=True) True

with torch.no_grad():
    a[:] = 10.
print(a, a.is_leaf)
# tensor([10., 10., 10., 10.], requires_grad=True) True

loss = (a*a).mean()
loss.backward()
print(a.grad)
# tensor([5., 5., 5., 5.])


In [17]:
'''detach '''
a = torch.tensor([7., 0, 0], requires_grad=True)
b = a + 2
print(b)
# tensor([9., 2., 2.], grad_fn=<AddBackward0>)

loss = torch.mean(b * b)

b_ = b.detach() # 可以得到 tensor的数据 + requires_grad=False 的版本
b_.zero_()
print(b)
# tensor([0., 0., 0.], grad_fn=<AddBackward0>)
# 储存空间共享，修改 b_ , b 的值也变了



tensor([9., 2., 2.], grad_fn=<AddBackward0>)
tensor([0., 0., 0.], grad_fn=<AddBackward0>)


In [ ]:
#关于使用 GPU 还有一个点，在我们想把 GPU tensor 转换成 Numpy 变量的时候，需要先将 tensor 转换到 CPU 中去，因为 Numpy 是 CPU-only 的。
# x  = torch.rand([3,3], device='cuda')
# x_ = x.cpu().numpy()

#其次，如果 tensor 需要求导的话，还需要加一步 detach，再转成 Numpy 。
# y  = torch.rand([3,3], requires_grad=True, device='cuda')
# y_ = y.cpu().detach().numpy()

In [21]:
''' tensor.item()'''
# item() 只适用于 tensor 只包含一个元素的时候。因为大多数情况下我们的 loss 就只有一个元素，所以就经常会用到 loss.item()。
x  = torch.randn(1, requires_grad=True, device='cpu')
print(x)
y = x.item()
print(y, type(y))


# 如果想把含多个元素的 tensor 转换成 Python list 的话，要使用 tensor.tolist()
x = torch.randn([2, 2])
y = x.tolist()
print(y)

tensor([0.4484], requires_grad=True)
0.4484216272830963 <class 'float'>
[[-0.9549000859260559, 1.7563014030456543], [-0.007843704894185066, -1.2936357259750366]]
